# 2º Trabalho de Inteligência Artificial
##### Trabalho Realizado por Francisco Tavares, Rodrigo Batista e Rodrigo Taveira

## Índice

1. [Introdução](#Introdução)

2. [Objetivo](#Objetivo)

3. [Funções auxiliares](#Funções-auxiliares)

4. [Construção da árvore](#Construção-da-árvore)

5. [Menus](#Menus)

6. [Divisão dos dados](#Divisão-dos-dados)

7. [Previsão de dados](#Previsão-de-dados)

8. [Datasets](#Datasets)

9. [Testes e resultados](#Testes-e-resultados)

10. [Datasets](#Datasets)

11. [Conclusão](#Conclusão)

## Introdução

Este trabalho tem como objetivo explorar e implementar um algoritmo de árvores de decisão, baseado no conceito apresentado no livro "Artificial Intelligence: a Modern Approach" de Stuart Russell e Peter Norvig.
Será adoptado um algoritmo semelhante ao ID3, que utiliza a entropia como medida de seleção de atributos para construir a árvore de decisao.

Além disso, este trabalho abrange a implementação do algoritmo em diferentes conjuntos de dados. Esta abordagem permite explorar a eficácia e versatilidade da árvore de decisão em diferentes contextos

## Objetivo

O objetivo deste trabalho é estudar e compreender a implementação de uma decision tree. Também iremos estudar a influência de determinados parâmetros na construção das mesmas bem como a importância de resolver problemas tal como o class imbalance

In [ ]:
import pandas as pd
from collections import Counter
import numpy as np
from scrapy.spidermiddlewares import depth
import random 
import copy
import math

from Trabalho_IA_versao_DECISION_TREE import ConnectFour

## Funções auxiliares

[Go back to the top](#Índice)

-Função __choose_best_attribute__(attributes, examples): Esta função com base na entropia e na informação ganha calcula o melhor atributo.

-Função __plurality_value__(examples): Esta função em termos de implementação, começa por fazer uma contagem das ocorrências das classes, onde depois o máximo desta contagem é determinado usando a função 'max()'. Se houver mais do que uma classe com a mesma contagem máxima, a função escolhe aleatoriamente uma das classes.

-Função __calculate_attribute_entropy__(examples, attribute): Esta função calcula a entropia de um atributo

-Função __entropy__(class1): Esta função é responsável por calcular a entropia.

-Função __calculate_numeric_splits__(examples,attribute): Esta função está responsável por calcular o valor que melhor separa o dataset para os atributos númericos. Faz isso através do cálculo da informação ganha.

-Função __information_gain__(parent_class,class1,class2): Esta função calcula o ganho de informação. Usa a função __calculate_entropy__.

-Função __attribute_values__(attribute, dataframe): Nesta função são retornados todos os valores únicos de um atributo

-Função __transform_boolean__(examples): Transforma todos os valores booleanos 'True' e 'False' em 'Yes' e 'No'.

-Função __subset_with_attribute_value__(attribute, value, dataframe): Cria um dataset com os dados com valor igual ao value inserido.

-Função __subset_with_attribute_value_numeric__(attribute, value, operator, examples): Cria um dataset com dados "<=" ou ">" que o valor dado, dependendo do operator inserido.

-Função __all_same_class__(examples): Esta função retorna True se todas as entradas forem da mesma classe. Retorna False caso haja, pelo menos, um diferente.

-Função __print_decision_tree__(tree, depth=0): Esta função é responsável por imprimir a árvore no formato pedido pelo professor.

-Função __prever_classe__(arvore, caminho): Nesta função é usado a árvore e o caminho gerado pela função __caminho_dados__ para prever a classe de um determinado dado.

-Função __caminho_dados__(instancia, arvore): Esta função percorre a árvore (dicionário) e retorna o caminho de uma determinada entrada do dataset. Esta função consegue lidar com dados numéricos e strings.

-Função __prever_dataset__(df, arvore): Esta função prevê a classe de todas as entradas de um dataset. Para isso percorre todas as linhas e usa as funções __caminho_dados__ e __prever_classe__.

-Função __accuracy__(dataset,previsoes): Esta função retorna-nos a accuracy. Se a classe prevista for igual à classe "real" adiciona mais um ao contador e ao total, caso não seja soma apenas ao total. 

-Função __predict_new_example__(dt,new_example): Esta função está responsável por adicionar um novo exemplo ao dataset para poder ser avaliado. Começa por separar todos os valores pelas vírgulas e por anotar os tipos do dataset num vetor. De seguida percorre todos os tipos e caso seja um inteiro ou um float transforma os dados lidos no respetivo tipo. Por fim acrescenta a nova linha ao dataset e retorna o dataset. 

-Função __over_sample__(df): Esta é uma função de pré processamento responsável por aumentar a classe em menor número em datasets com classes desbalanceadas (neste caso o dataset 2).

In [ ]:
import math
from sklearn.utils import resample

def choose_best_attribute(attributes, examples):

    entropy = calculate_entropy(examples[examples.columns[-1]].tolist())
    infos_gain = []
      
    for attribute in attributes:
        entropy_attribute=calculate_attribute_entropy(examples, attribute)
        info_gain=entropy - entropy_attribute
        infos_gain.append(info_gain)
    
    maximo=float('-inf')
    indice=0
    for i in range (len(infos_gain)):
        if infos_gain[i]>maximo:
            maximo=infos_gain[i]
            indice=i
    
    best_attribute=attributes[indice]
    
    return best_attribute
    

def plurality_value(examples):
    counts = {}
    for i in range(examples.shape[0]):
        classification = examples.iloc[i, -1] 
        counts[classification] = counts.get(classification, 0) + 1

    max_count = max(counts.values())
    common_values = [value for value, count in counts.items() if count == max_count]

    return random.choice(common_values)
    
def calculate_attribute_entropy(examples,attribute):
    
    values=attribute_values(attribute,examples)
    entropy_attribute=0
    
    for value in values:
        subset = subset_with_attribute_value(attribute, value, examples)
        subset_labels = subset[examples.columns[-1]]
        subset_entropy = calculate_entropy(subset_labels)
        subset_probability = subset.shape[0]/examples.shape[0]
        entropy_attribute +=subset_probability * subset_entropy
        
    return entropy_attribute

def calculate_entropy(class1):
    
    label_counts = Counter(class1)
    total_samples = len(class1)
    entropy = 0
    
    for count in label_counts.values():
        probability = count / total_samples
        entropy -= probability * math.log2(probability)
    
    return entropy

def calculate_numeric_splits(examples,attribute):
    
    values=attribute_values(attribute,examples)
    best_split= None
    best_info_gain= float('-inf')
    
    if len(values) == 1:
        return values[0]
    
    for value in values:
        subset1 = subset_with_attribute_value_numeric(attribute, value, "<= ", examples)
        subset2 = subset_with_attribute_value_numeric(attribute, value, "> ", examples)
        
        labels1 = subset1[examples.columns[-1]]
        labels2 = subset2[examples.columns[-1]]

        info_gain = information_gain(examples[examples.columns[-1]], labels1, labels2)

        if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_split = value

    return best_split

def information_gain(parent_class,class1,class2):
    parent_entropy = calculate_entropy(parent_class)
    weight1 = len(class1) / len(parent_class)
    weight2 = len(class2) / len(parent_class)
    entropy1 = calculate_entropy(class1)
    entropy2 = calculate_entropy(class2)
    info_gain = parent_entropy - (weight1 * entropy1) - (weight2 * entropy2)
    return info_gain

def attribute_values(attribute, dataframe):
    return dataframe[attribute].unique()

def transform_boolean(examples):
    examples.replace({True: "Yes", False: "No"}, inplace=True)

def subset_with_attribute_value(attribute, value, dataframe):
    subset = dataframe[dataframe[attribute] == value]
    #print(subset)
    return subset

def subset_with_attribute_value_numeric(attribute, value, operator, examples):
    if operator == "<= ":
        subset = examples[examples[attribute] <= value]
    elif operator == "> ":
        subset = examples[examples[attribute] > value]    
    return subset

def all_same_class(examples):
    for i in range (examples.shape[0]):
        example= examples.iloc[i,-1]
        if example != examples.iloc[0,-1]:
            return False
    return True

def print_decision_tree(tree, depth=0):
    if isinstance(tree, dict):
        for attribute, subtree in tree.items():
            print("  " * depth + "<" + attribute + ">")
            for value, subtree_or_class in subtree.items():
                if isinstance(subtree_or_class, dict):
                    print("  " * (depth + 1) + value + ":")
                    print_decision_tree(subtree_or_class, depth + 2)
                else:
                    if "(" in subtree_or_class:
                        counter=0
                        class_, counter = subtree_or_class.split(" (")
                        class_ = class_.strip()
                        counter = counter.replace(")", "").strip()
                        print("  " * (depth + 1) + value + ": " + class_ + " (" + counter + ")")
                    else:
                        print("  " * (depth + 1) + value + ": " + subtree_or_class)
    else:
        print("  " * depth + tree)

def prever_classe(arvore, caminho):
    no_atual = arvore
    
    for passo in caminho:
        if isinstance(no_atual, dict) and passo in no_atual:
            no_atual = no_atual[passo]
        else:
            return "Caminho não encontrado na árvore"
    
    if isinstance(no_atual, str):
        classe = no_atual.split(' ')[0]
        return classe
    else:
        return "Caminho não leva a uma classe final"

def caminho_dados(instancia, arvore):
    caminho = []
    no_atual = arvore
    while isinstance(no_atual, dict):
        atributo = next(iter(no_atual))
        if atributo not in instancia:
            print("Atributo:", atributo)
            return None  # Atributo não encontrado na instância
        valor = instancia[atributo]
        
        for chave in no_atual[atributo]:
            if chave.startswith('<= ') or chave.startswith('> '):
                comparacao, limite = chave.split(' ')
                limite = float(limite)
                if (comparacao == '<=' and valor <= limite) or (comparacao == '>' and valor > limite):
                    caminho.append(atributo)
                    caminho.append(chave)
                    no_atual = no_atual[atributo][chave]
                    break
            elif valor == chave:
                caminho.append(atributo)
                caminho.append(chave)
                no_atual = no_atual[atributo][chave]
                break
        else:
            return None  # Caminho não encontrado para o valor do atributo
    return caminho

def prever_dataset(df, arvore):
    previsoes = []
    for _, instancia in df.iterrows():
        caminho = caminho_dados(instancia, arvore)
        if caminho is None:
            previsoes.append("Caminho não encontrado na árvore")
        else:
            classe = prever_classe(arvore, caminho)
            previsoes.append(classe)
    return previsoes
    

def accuracy(dataset,previsoes):
    cont=0
    total=0
    for i in range (len(previsoes)):
        if (previsoes[i] == dataset.iloc[i,-1]):
            cont+=1
            total+=1
        else:
            total+=1
    return cont/total

def predict_new_example(dt,new_example):

    values = [valor.strip() for valor in new_example.split(',')]

    tipos=dt.dtypes

    for i in range (len(values)):
        if tipos[i]=='int64':
            values[i]=int(values[i])
        if tipos[i]=='float64':
            values[i]=float(values[i])


    dt1 = pd.DataFrame(values)
    dados_nova_linha = {nome_coluna: [valor] for nome_coluna, valor in zip(dt.columns, values)}
    dt= pd.concat([dt, pd.DataFrame(dados_nova_linha)], ignore_index=True)
    return dt

def over_sample(df):
    last_column=df.columns[-1]
    valor = df[last_column].mode()[0]
    df_majority = df[df[last_column] == valor]
    df_minority = df[df[last_column] != valor]
    
    df_minority_upsampled = resample(df_minority, 
                                     replace=True,     
                                     n_samples=len(df_majority),    
                                     random_state=123) 

    df_upsampled = pd.concat([df_majority, df_minority_upsampled])
    
    return df_upsampled

## Construção da árvore

[Go back to the top](#Índice)

Esta é a função responsável por gerar a árvore. Seguindo o manual da disciplina, esta função recursiva escolhe o melhor atributo através da função __choose_best_attribute__. De seguida é criado um nó com esse atributo e guardado em duas variáveis os valores únicos restantes desse atributo e todos os valores únicos desse atributo no ínicio do dataset.
De seguida é verificado se os valores são inteiros/floats. Caso sejam, estes dados são lidados de maneira diferente, através da definição de intervalos. É calculado o melhor valor para dividir os dados e depois são divididos em <= e > que esse valor. Após isso é chamada novamente a função onde são passados todos os atributos menos aquele já usado.
Caso não sejam inteiros/float, são percorridos todos os valores e os dados divididos baseando-se nesse valor. Caso não haja mais exemplos retorna a classe mais comum. De realçar também o caso onde são "perdidos" valores e por isso tivemos de fazer uma condição para adicionar esse valor à árvore com contador igual a 0.

Falando brevemente dos caso base, o primeiro caso refere-se a não haver mais dados, então é retornada a classe com maior número. O segundo refere-se a todos os dados serem da mesma classe e o terceiro a não haver mais atributos.

In [ ]:
def learn_decision_tree(examples, attributes, parent_examples):
    if len(examples) == 0:
        class_ = plurality_value(parent_examples)
        counter = len(examples)
        return f"{class_} ({counter})"
    
    if all_same_class(examples):
        class_ = examples.iloc[0,-1]
        counter = len(examples)
        return f"{class_} ({counter})"
    
    if len(attributes) == 0:
        class_ = plurality_value(examples)
        counter = len(examples)
        return f"{class_} ({counter})"
    
    best_attribute = choose_best_attribute(attributes, examples)
    tree = {best_attribute: {}}
    
    values = attribute_values(best_attribute, examples)
    all_attribute_values = set(attribute_values(best_attribute, dt))
    set_values=set(values)
    
    if isinstance(values[0], (int, float, np.int64)):
        best_split = calculate_numeric_splits(examples, best_attribute)
        
        subset1 = subset_with_attribute_value_numeric(best_attribute, best_split, "<= ", examples)
        subset2 = subset_with_attribute_value_numeric(best_attribute, best_split, "> ", examples)
        
        subtree1 = learn_decision_tree(subset1, [attr for attr in attributes if attr != best_attribute], examples)
        subtree2 = learn_decision_tree(subset2, [attr for attr in attributes if attr != best_attribute], examples)
        
        tree[best_attribute]["<= " + str(best_split)] = subtree1
        tree[best_attribute]["> " + str(best_split)] = subtree2
    else:
        for class_value in all_attribute_values - set_values:
            class_=examples.iloc[0,-1]
            tree[best_attribute][class_value] = f"{class_} (0)"
        for value in values:
            subset = subset_with_attribute_value(best_attribute, value, examples)
            if len(subset) == 0:
                subtree = plurality_value(examples)
            else:
                subtree = learn_decision_tree(subset, [attr for attr in attributes if attr != best_attribute], examples)
            tree[best_attribute][value] = subtree
    
    return tree


## Menus

[Go back to the top](#Índice)

Neste pedaço de código estão presentes as funções correspondentes aos menus

In [ ]:
from IPython.display import clear_output

def menu_1():
    print("Welcome to Decision Tree Classifier!")
    print("Choose between each type of data")
    print("1. Full set")
    print("2. Train/test set")

    choice = input("Enter your choice: ")
    tamanho=0.0
    div=False
    
    if choice == '2':
        
        tamanho=float(input("Enter the test data size (0.0 to 0.9): "))
        
        div=True
        
    dt,ID_dataset=menu_2()
    
    return dt,div,tamanho,ID_dataset

def menu_2():
    print("Choose your dataset")
    print("1. Restaurant")
    print("2. Weather")
    print("3. Iris")
    print("4. ConnectFour")
    
    choice = input("Enter the dataset number: ")
    
    ID_dataset=0
    
    if choice == '1':
        dt=pd.read_csv('restaurant.csv')
        dt.drop(dt.columns[0], axis=1, inplace=True)
        ID_dataset=1
        
    elif choice == '2':
        dt=pd.read_csv('weather.csv')
        dt.drop(dt.columns[0], axis=1, inplace=True)
        transform_boolean(dt)
        ID_dataset=2
        
    elif choice == '3': 
        dt=pd.read_csv('iris.csv')
        dt.drop(dt.columns[0], axis=1, inplace=True)
        ID_dataset=3
        
    elif choice == '4':
        dt=pd.read_csv('connect4.csv')
        ID_dataset=4
    
    transform_boolean(dt)
    
    return dt, ID_dataset
        

## Divisão dos dados

[Go back to the top](#Índice)

Este código destina se à chamada do menu, onde é possível escolher se há ou não divisão de dados (em caso afirmativo podemos escolher o tamanho dos dados de teste) e qual dataset se pretende usar (dentro dos disponibilizados pelo professor)

Como output recebemos a árvore do dataset correspondente e em caso de divisão de dados recebemos também a accuracy

In [ ]:
import random
from sklearn.model_selection import train_test_split


dt,div,tamanho,ID_dataset= menu_1()

#dt=pd.read_csv('nomedoficheiro.csv')         #Aqui é possível acrescentar um novo dataset para ser gerada uma árvore a partir dele
#dt.drop(dt1.columns[0], axis=1, inplace=True)   #Se tiver uma coluna com índices usar este comando

class_imbalance =input("Quer aplicar uma solução para class imbalance? (y/n)")

if class_imbalance == 'y':
    dt= over_sample(dt)

if div:
    train_set, test_set = train_test_split(dt, test_size=tamanho, random_state=42)
    tree = learn_decision_tree(train_set,dt.columns[:-1], None)
    print("Árvore de Decisão:")
    print_decision_tree(tree)

    previsoes = prever_dataset(test_set, tree)
    acc=accuracy(test_set, previsoes)
    print()
    print("Accuracy: ",acc)
    
else:
    tree = learn_decision_tree(dt,dt.columns[:-1], None)
    #print(tree)
    print("Árvore de Decisão:")
    print_decision_tree(tree)
    previsoes = prever_dataset(dt, tree)
    acc=accuracy(dt, previsoes)
    print()
    print("Accuracy: ",acc)

## Previsão de dados

[Go back to the top](#Índice)

### Apenas uma linha


Esta secção dedica-se à previsão de apenas um exemplo (inserido pelo utilizador)

In [ ]:
resp= input("Insira o novo exemplo: ")

dt = predict_new_example(dt, resp)

previsao = prever_dataset(dt, tree)
previsao_final = previsao[len(previsao) - 1]


print("Previsão: ", previsao_final)

### Dataset


Nesta secção é possível inserir um ficheiro para ser avaliado. Para isso basta mudar 'nomedoficheiro' para o caminho onde se encontra o ficheiro e executar o código. Após a execução teremos a accuracy bem como as previsões de cada entrada do dataset.

### Connect Four

#### Funções relacionadas ao dataset 4

-Função __transform_connect_four__(state): Esta função está responsável por receber um estado em forma de matriz e transformá-lo numa linha com as características do dataset 'connectfour'. É essencial para podermos prever o resultado de um estado.

-Função __predict_move_tree__(dt,state): Esta função através do dataset recebido e do estado atual do jogo, está responsável por prever o melhor movimento e executá-lo. Gera os sucessores e faz a previsão final de cada um. Se for "win", o movimento é armazenado num vetor, onde no fim se este não estiver vazio, será feito um random desses movimentos. Caso seja vazio, significa que não houve nenhum resultado "win" e então é feito um movimento aleatório.

In [ ]:
def transform_connect_four_to_dataset(state):
    resp=""
    linha=0
    coluna=0
    for i in range (42):
        if linha == 6:
            linha = 0
            coluna += 1
        if state[5-linha][coluna] == '-':
            state[5-linha][coluna] = 'b'
        elif state[5-linha][coluna] == 'X':
            state[5-linha][coluna] = 'x'
        elif state[5-linha][coluna] == 'O':
            state[5-linha][coluna] = 'o'

        resp = resp + state[5-linha][coluna] + ","
        linha+=1
        
    resp = resp.rstrip(',')
    return resp


In [ ]:
def predict_move_tree(dt,state):
    
    #print("Estado: ",state)
    
    sucessores, cols = state.successors()

    movimentos=[]

    for i in range(len(sucessores)):
        
        if sucessores[i] is None:
            continue
                
        resp=transform_connect_four_to_dataset(sucessores[i].board)
        
        dt = predict_new_example(dt, resp)
        previsao = prever_dataset(dt, tree)
        previsao_final = previsao[len(previsao) - 1]
        if previsao_final == "win":
            movimentos.append(cols[i])

    if movimentos == []:
        possible_moves = [col for col in range(7) if not state.is_full(col)]
        if possible_moves:
            random_move = random.choice(possible_moves)  # Escolhe um movimento possível aleatório
            state.move(random_move, state.turn)
    else:
        state.move(random.choice(movimentos), state.turn)
        
        

game=ConnectFour()

game.board = [["-"] * 7 for _ in range(6)]


while not (game.winner or game.is_board_full()):
    if game.turn=="X":
        predict_move_tree(dt,game)
    else:
        alpha = float('-inf')
        beta = float('inf')
                    
        _,prox_move=game.alphabeta(5,alpha,beta,1)
        game.move(prox_move,game.turn)
    
        game.check_winner()
        print (game)


if game.winner:
    print("Game Over! Player", game.player_winner, "wins!!")
else:
    print("Game Over! It's a draw")
        

## Datasets

[Go back to the top](#Índice)

#### Restaurant

Este conjunto de dados contém informações sobre clientes e restaurantes, incluindo tipo de comida, tempo de espera, preço, etc. A variável de classe indica se o cliente irá ou não esperar para comer no restaurante.

Este dataset serviu como dataset inicial para o desenvolvimento deste projeto, não apresentando grandes dificuldades.


In [28]:
dt=pd.read_csv('restaurant.csv')
dt.head()

,ID,Alt,Bar,Fri,Hun,Pat,Price,Rain,Res,Type,Est,Class
0,X1,Yes,No,No,Yes,Some,$$$,No,Yes,French,0-10,Yes
1,X2,Yes,No,No,Yes,Full,$,No,No,Thai,30-60,No
2,X3,No,Yes,No,No,Some,$,No,No,Burger,0-10,Yes
3,X4,Yes,No,Yes,Yes,Full,$,No,No,Thai,10-30,Yes
4,X5,Yes,No,Yes,No,Full,$$$,No,Yes,French,>60,No


#### Weather

Este conjunto de dados contém informações sobre condições climáticas para jogar tênis. O objetivo é ensinar uma árvore de decisão para que possa decidir quais são as melhores condições para jogar tênis.

Este dataset já apresenta dados numéricos bem como dados booleanos, os quais tivemos de fazer alterações para que o nosso algoritmo funcionasse. Vimos também que apresentava um problema de class imbalance, ou seja, uma das classes (neste caso a class 'Yes') é superior à outra (neste caso a class 'No').

In [29]:
dt=pd.read_csv('weather.csv')
dt.head()

,ID,Weather,Temp,Humidity,Windy,Play
0,1,sunny,85,85,False,no
1,2,sunny,80,90,True,no
2,3,overcast,83,86,False,yes
3,4,rainy,70,96,False,yes
4,5,rainy,68,80,False,yes


#### Iris

Este conjunto de dados contém informações numéricas sobre plantas de três classes: iris setosa, iris virginica e iris versicolor. Os atributos são comprimento e largura da pétala e comprimento e largura da sépala. O objetivo é ensinar uma árvore de decisão para que possa determinar qual classe uma planta pertence, dadas as suas medidas de sépala e pétala. 

Este dataset apresenta apenas dados numéricos e uma classe com três valores diferentes. Em relação aos restantes este dataset também apresenta mais entradas.

In [30]:
dt=pd.read_csv('iris.csv')
dt.head()

,ID,sepallength,sepalwidth,petallength,petalwidth,class
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


#### Connect Four

Este é um conjunto de dados onde cada linha corresponde a uma configuração de tabuleiro do jogo connect four. O objetivo é induzir uma árvore de decisão para este conjunto de dados e através da mesma prever onde jogar na próxima jogada.

Este dataset apresenta muitas entradas (cerca de 67000)

Usando este dataset, iremos recorrer às funções do primeiro trabalho para colocar o AlphaBeta a jogar contra o DecisionTree.

In [32]:
dt=pd.read_csv('connect4.csv')
dt.head()

,a1,a2,a3,a4,a5,a6,b1,b2,b3,b4,...,f4,f5,f6,g1,g2,g3,g4,g5,g6,class
0,b,b,b,b,b,b,b,b,b,b,...,b,b,b,b,b,b,b,b,b,win
1,b,b,b,b,b,b,b,b,b,b,...,b,b,b,b,b,b,b,b,b,win
2,b,b,b,b,b,b,o,b,b,b,...,b,b,b,b,b,b,b,b,b,win
3,b,b,b,b,b,b,b,b,b,b,...,b,b,b,b,b,b,b,b,b,win
4,o,b,b,b,b,b,b,b,b,b,...,b,b,b,b,b,b,b,b,b,win


## Testes e Resultados

[Go back to the top](#Índice)

Nesta seccção estarão presentes testes onde foram alterados alguns parâmetros e iremos analisar e discutir os resultados obtidos

### Comparação de árvores com e sem divisão dos dados

Neste ponto iremos comparar as árvores geradas sem divisão de dados e as árvores geradas com 0.7 de tamanho de dados de treino para os primeiros três datasets.

##### S/ divisão

##### C/ divisão

A accuracy depois de dividir os dados em treino e teste foi de 75%.

##### S/ divisão

##### C/ divisão

A accuracy depois de dividir os dados em treino e teste foi de 40%

Através dos resultados obtidos, podemos verificar diferenças nas árvores geradas dos dois primeiros datasets, sendo o segundo mais evidente. Como há menos dados a serem usados para construir a árvore de decisão, isso significa menos informação o que pode levar a decisões "precipitadas" da árvore.

##### S/ divisão

##### C/ divisão

Neste caso a accuracy subiu de 97% para 100%. Isto pode dever-se ao facto de quando usamos a divisão dos dados, haver menos para avaliar e por essa razão a accuracy subir para 100%. Outra razão poderá ser a redução de overfitting já que são usados menos dados para construir a árvore.

### Comparação entre os diferentes tamanhos de teste e treino

Neste ponto irá ser estudada a influência dos diferentes tamanhos de teste e treino na construção da árvore de treino.

#### Restaurant

##### Tamanho de teste = 0.2 // 67%

##### Tamanho de teste = 0.4 // 80%

##### Tamanho de teste = 0.6 // 63%

Nestes exemplos conseguimos ver um aumento e uma descida na accuracy. Quando o tamanho de teste é "pequeno" provavelmente ocorre overfitting já que a accuracy desceu em relação ao tamanho de teste 0.4. Quando o tamanho de teste é demasiado grande, não há dados suficientes para treinar a árvore e nesse caso a accuracy também é bastante baixa.

#### Weather

##### Tamanho de teste = 0.2 // 33%

##### Tamanho de teste = 0.4 // 50%

##### Tamanho de teste = 0.6 // 44%

Como no exemplo anterior, as accuracys subiram e desceram devido às mesmas razões. Podemos ver que quando o tamanho de testes é baixo (e consequentemente o de treino é alto) a árvore é mais complexa o que leva a overfitting. Já quando é baixo, a árvore é demasiado simples o que leva também a uma accuracy baixa.

#### Iris

##### Tamanho de teste = 0.2 // 100%

##### Tamanho de teste = 0.4 // 98%

##### Tamanho de teste = 0.6 // 91%

Neste caso, ao contrário dos anteriores, à medida que o tamanho de teste aumenta a accuracy diminui. Isso deve-se ao facto de, como o dataset é mais complexo e maior, os dados de treino irão ser mais e por essa razão a árvore ficará mais robusta e por consequência aumenta a accuracy.

#### Connect Four

##### Tamanho de teste = 0.2 // 75%

##### Tamanho de teste = 0.4 // 73%

##### Tamanho de teste = 0.6 // 72%

Por último, fizemos com o quarto dataset onde obtivemos accuracys semelhantes em todos os testes, o que é esperado devido à dimensão do dataset e à semelhança entre os dados.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

datasets = ['Restaurante', 'Weather', 'Iris', 'Connect4']
divisions = ['0.2', '0.4', '0.6']
values1 = [67, 80, 63, 33, 50, 44, 100, 98, 91, 75, 73, 72]

x = np.arange(len(datasets))
width = 0.2 

fig, ax = plt.subplots()

bars1 = ax.bar(x - width, values1[0::3], width, label='0.2', color='turquoise')
bars2 = ax.bar(x, values1[1::3], width, label='0.4', color='indigo')
bars3 = ax.bar(x + width, values1[2::3], width, label='0.6', color='orange')

ax.set_xlabel('Datasets')
ax.set_ylabel('Accuracy (%)')
ax.set_title('Diferentes tamanhos de divisões')
ax.set_xticks(x)
ax.set_xticklabels(datasets)
ax.legend()

plt.ylim(0, 100)
plt.show()


### Over-Sampling

Neste teste decidimos ajustar melhor os dados do dataset 'Weather' que apresentava dados com a classe desbalanceada, isto é, possuia mais dados com classe 'Yes' do que dados com classe 'No'.
Vamos verificar se teve algum efeito na accuracy

#### Restaurant

##### Tamanho de teste = 0.2 // 30%

##### Tamanho de teste = 0.4 // 60%

##### Tamanho de teste = 0.6 // 50%

Como podemos observar, houve uma diminuição bastante acentuada nas accuracys em todos os testes, logo concluimos que este dataset não sofre do problema de class imbalance.

#### Weather

##### Tamanho de teste = 0.2 // 25%

##### Tamanho de teste = 0.4 // 75%

##### Tamanho de teste = 0.6 // 36%

Através dos resultados obtidos, vemos que houve um aumento da accuracy. Com isto podemos afirmar que este dataset sofria bastante do problema de class imbalance, no entanto após a implementação da nossa solução, a accuracy subiu para um número considerável.

#### Iris

Após a execução dos testes, os resultados não se alteraram. Esta solução implementada não se mostrou relevante para este dataset, concluindo assim que este dataset não sofre de class imbalance.

#### Connect Four

##### Tamanho de teste = 0.2 // 88%

##### Tamanho de teste = 0.4 // 84%

##### Tamanho de teste = 0.6 // 79%

Os resultados mostram um aumento significativo na accuracy após a implementação desta solução para lidar com o class imbalance. Isso sugere que o dataset inicialmente sofria do problema de class imbalance, mas com a aplicação da nossa solução, a accuracy melhorou consideravelmente.

In [ ]:
values2 = [30, 60, 50, 25, 75, 36, 100, 98, 91, 88, 84, 79]

fig, ax = plt.subplots()

bars1 = ax.bar(x - width, values2[0::3], width, label='0.2', color='turquoise')
bars2 = ax.bar(x, values2[1::3], width, label='0.4', color='indigo')
bars3 = ax.bar(x + width, values2[2::3], width, label='0.6', color='orange')

ax.set_xlabel('Datasets')
ax.set_ylabel('Accuracy (%)')
ax.set_title('Diferentes tamanhos de divisões com over-sampling')
ax.set_xticks(x)
ax.set_xticklabels(datasets)
ax.legend()

plt.ylim(0, 100)
plt.show()


### Comparação entre AlphaBeta e DecisionTree

Neste último teste, abordaremos o último dataset, onde este será responsável por decidir onde o jogador "X" irá colocar a sua peça. Para este teste decidimos usar o AlphaBeta como jogador "O" que foi implementado no trabalho anterior.

É de esperar que o AlphaBeta ganhe as 10 partidas pois o DecisionTree não é adequado para este tipo de problemas.

Como podemos observar, os resultados foram de encontro aos esperados, já que o DecisionTree em caso de ter vários estados com classe "win" é escolhido aleatoriamente um movimento.

## Conclusão

Com este trabalho conseguimos aprender um pouco mais sobre a implementação e o desenvolvimento de uma DecisionTree. Ao longo do nosso trabalho fomos encontrando alguns problemas nos datasets, os quais resolvemos adaptando o nosso algoritmo aos mesmos.

Decidimos ainda fazer alguns testes para compararmos alguns parâmetros e vermos a influência de cada um na accuracy da árvore gerada. Observamos que a divisão do dataset em treino e teste alterou a accuracy. Mudando o tamanho do teste observamos também descidas e subidas na accuracy final.
Em termos de pré processamento, para além das funções iniciais, implementamos também uma solução para o problema de class imbalance que se mostrou bastante eficaz na redução deste problema no dataset 2 e 4.

Por fim, decidimos testar o DecisionTree contra o algoritmo AlphaBeta e este, como era de esperar, perdeu as 10 partidas.